In [36]:
import numpy as np
import cv2
import os

In [37]:
def rescaleFrame(frame, scale=0.5):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)
    return cv2.resize(frame, dimensions, interpolation=cv2.INTER_AREA)

def calculate_brightness(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the mean brightness of the image
    brightness = np.mean(gray)

    return brightness

In [38]:
def find_parallel(lines):
    max_parallel_count = 0
    best_line_index = -1
    best_parallel_indices = []

    for i in range(len(lines)):
        parallel_indices = []

        for j in range(len(lines)):
            if i == j:
                continue

            # Extract line coordinates
            x1_i, y1_i, x2_i, y2_i = lines[i][0]
            x1_j, y1_j, x2_j, y2_j = lines[j][0]

            # Check if the lines are approximately parallel
            delta_x_i = x2_i - x1_i
            delta_y_i = y2_i - y1_i
            delta_x_j = x2_j - x1_j
            delta_y_j = y2_j - y1_j

            # Avoid division by zero
            if delta_x_i == 0 or delta_x_j == 0:
                continue

            slope_i = delta_y_i / delta_x_i
            slope_j = delta_y_j / delta_x_j
 
            if abs(slope_i - slope_j) < 0.05:
                parallel_indices.append(j)

        if len(parallel_indices) > max_parallel_count:
            max_parallel_count = len(parallel_indices)
            best_line_index = i
            best_parallel_indices = parallel_indices

    return best_line_index, best_parallel_indices

In [39]:
def detectBarcodes(image, showProcess=False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.medianBlur(gray, 1) 

    ddepth = cv2.CV_32F
    gradX = cv2.Sobel(blurred, ddepth=ddepth, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(blurred, ddepth=ddepth, dx=0, dy=1, ksize=-1)
    # subtract the y-gradient from the x-gradient
    
    edgeDetection = cv2.subtract(gradX, gradY)
    edgeDetection = cv2.convertScaleAbs(edgeDetection)
    
    cv2.imshow("Edge", edgeDetection)

    kernel = np.ones((3, 3), np.uint8)
    
    _, thresh = cv2.threshold(edgeDetection, 150, 255, cv2.THRESH_BINARY)
    cv2.imshow("Thresh", thresh)
    erode = cv2.erode(thresh, None, iterations= 1)
    dilate = cv2.dilate(erode, None, iterations= 1)
    
    cv2.imshow("Morphology", dilate)
    
    lines = cv2.HoughLinesP(image=erode,rho=1.25,theta=np.pi/180, threshold=100,lines=np.array([]), minLineLength=100, maxLineGap=10)

    blank = np.zeros(image.shape, dtype=np.uint8)
    
    blank_temp = blank.copy()
    # a = lines.shape[0]
    if lines is None:
        cv2.imshow("NO LINES", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
    
    for i in range(len(lines)):
        cv2.line(blank_temp, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0, 0, 255), 3, cv2.LINE_AA)
        
    cv2.imshow("All Lines", blank_temp)
    
    copy = image.copy()
    
    if lines is None or len(lines) < 10:
        return False
    
    best_line_index, best_parallel_indices = find_parallel(lines)

    if best_line_index != -1:
        best_line = lines[best_line_index][0]
        print(f"Best line with index {best_line_index} has {len(best_parallel_indices)} parallel lines.")
    
        # Draw the best line
        cv2.line(blank, (best_line[0], best_line[1]), (best_line[2], best_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
    
        # Draw all parallel lines
        for parallel_index in best_parallel_indices:
            parallel_line = lines[parallel_index][0]
            cv2.line(blank, (parallel_line[0], parallel_line[1]), (parallel_line[2], parallel_line[3]), (0, 0, 255), 3, cv2.LINE_AA)
    else:
        print("No parallel lines found.")
                
    if showProcess:
        # cv2.imshow("Masked Image", masked_image)
        # cv2.imshow("Image", image)
        cv2.imshow("Lines", blank)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    # imageName = image_path.split(os.path.sep)[-1]

In [40]:
folderPath = 'Images\Dataset'

# List all files in the folder
files = os.listdir(folderPath)

for file in files:
    # Check if the file is an image
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # Construct the full path to the image
        image_path = os.path.join(folderPath, file)

        # Read the image
        image = cv2.imread(image_path)

        if image.shape[0] > 800 or image.shape[1] > 1500:
            image = rescaleFrame(image)

        detectBarcodes(image, True)

Best line with index 24 has 53 parallel lines.
Best line with index 44 has 28 parallel lines.
Best line with index 4 has 49 parallel lines.
Best line with index 0 has 30 parallel lines.
Best line with index 40 has 6 parallel lines.
Best line with index 5 has 6 parallel lines.
Best line with index 27 has 56 parallel lines.
Best line with index 6 has 2 parallel lines.
Best line with index 2 has 7 parallel lines.
Best line with index 5 has 7 parallel lines.
Best line with index 16 has 14 parallel lines.
Best line with index 7 has 63 parallel lines.


KeyboardInterrupt: 